## 2 Ground Control Stations (simple)

This is a first version. The two ground control stations are working in series. 

In [1]:
from helpers import local2global, kill_processes,clean
from simulators import Simulator, Gazebo, ConfigGazebo, QGC
from plan import Plan
from oracle import Oracle,GCS
from config import Color
kill_processes()

In [2]:
# import random
# from helpers.change_coordinates import Offset
# n_vehicles = 10
# offsets = [
#         (
#             random.uniform(-5, 5),  # east
#             random.uniform(-5, 5),  # north
#             0,                          # up
#             random.randint(0, 359)      # heading
#         )
#         for _ in range(n_vehicles)
#     ]
# offsets

## Create Plans

In [3]:
offsets = [
    (-4.891311895481473, -1.3721148432366883, 0, 78),
    (4.272452965244035, -2.0624885894963674, 0, 308),
    (3.0267596011325804, 2.2013367285195207, 0, 21),
    (-2.0396000412483604, 0.07477394309030139, 0, 322),
    (-3.3237962685317655, 0.6696946809570994, 0, 289),
    (1.5219370395388898, -1.1094923308348625, 0, 149),
    (0.03170515670252261, 1.4467355333174172, 0, 17),
    (-3.167665911861357, -3.81136282344402, 0, 22),
    (-2.444501492007306, 2.3741511359188525, 0, 349),
    (0.48442199707763045, 4.043149308497993, 0, 38),
]
n_vehicles = len(offsets)

# offsets = [  # east, north, up, heading
#     (5., 5., 0., 90.),
#     (10., 0., 0., 45.),
#     (-5., -10., 0., 225.),
#     (-15., 0., 0., 0.),
#     (0., -20., 0., 0.),
# ]



local_paths = [Plan.create_square_path(side_len=8, alt=5) for _ in range(n_vehicles)]
plans = [Plan.basic(wps=path, wp_margin=0.5) for path in local_paths]
homes = [offset[:3] for offset in offsets]

## Visualization Parameters

In [4]:
green_nvehicles = 3
global_paths = [local2global(path, home) for path, home in zip(local_paths, homes)]
models = (n_vehicles-green_nvehicles)*[('iris',Color.BLUE)]+green_nvehicles*[('iris',Color.GREEN)]   # must agree with n_vehicles
markers = [ConfigGazebo.create_trajectory_from_array(array=global_paths[i],color=models[i][1]) for i in range(n_vehicles)]
Gazebo.plot_3d_interactive(markers=markers, title="Simulation Markers", frames=(0.2, 0.2, 0.6), ground=-0.05)


## Choose Simulator

In [5]:

gazebo_config = ConfigGazebo(world_path="simulators/gazebo/worlds/runway.world",models=models,markers = markers)
simulator = Gazebo(offsets=offsets,plans=plans,config=gazebo_config)

simulator = QGC(offsets=offsets, plans=plans, origin=(-35.3633245, 149.1652241, 0, 0))


simulator = Simulator(offsets=offsets, plans=plans)

In [6]:
#simulator._delete_all_qgc_links()

Launch Simulator

In [ ]:
sysids = list(range(1,n_vehicles+1))
simulator.launch()
orac = Oracle(sysids)
blue_gcs = GCS(sysids[:4],name='blue 🟦')
green_gcs = GCS(sysids[4:],name = 'green 🟩')

while len(orac.conns):
    blue_gcs.gather_broadcasts()
    green_gcs.gather_broadcasts()
    blue_gcs.save_pos()
    green_gcs.save_pos() 
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)
clean()

🚀 Vehicle 1 launched (PID 2903404)
🚀 Vehicle 1 logic launched (PID 2903405)
🚀 Vehicle 2 launched (PID 2903406)
🚀 Vehicle 2 logic launched (PID 2903407)
🚀 Vehicle 3 launched (PID 2903408)
🚀 Vehicle 3 logic launched (PID 2903409)
🚀 Vehicle 4 launched (PID 2903410)
🚀 Vehicle 4 logic launched (PID 2903411)
🚀 Vehicle 5 launched (PID 2903412)
🚀 Vehicle 5 logic launched (PID 2903413)
🚀 Vehicle 6 launched (PID 2903414)
🚀 Vehicle 6 logic launched (PID 2903415)
🚀 Vehicle 7 launched (PID 2903416)
🚀 Vehicle 7 logic launched (PID 2903417)
🚀 Vehicle 8 launched (PID 2903418)
🚀 Vehicle 8 logic launched (PID 2903419)
🚀 Vehicle 9 launched (PID 2903420)
🚀 Vehicle 9 logic launched (PID 2903421)
🚀 Vehicle 10 launched (PID 2903422)
🚀 Vehicle 10 logic launched (PID 2903423)
ℹ️  Running without visualization.
🔗 UAV logic 1 is connected to Oracle ⚪
🔗 UAV logic 2 is connected to Oracle ⚪
🔗 UAV logic 3 is connected to Oracle ⚪
🔗 UAV logic 4 is connected to Oracle ⚪
🔗 UAV logic 5 is connected to Oracle ⚪
🔗 UAV lo

## Print Positions gathered

In [ ]:
import numpy as np
from numpy.typing import NDArray
from typing import List
blue_paths:List[NDArray[np.float64]] = []
for sysid,path in blue_gcs.paths.items():
    blue_paths.append(np.array([x for x in path if x is not None]))

markers = [ConfigGazebo.create_trajectory_from_array(array=path,color=Color.BLUE) for path in blue_paths]
Gazebo.plot_3d_interactive(markers=markers,title="Simulation Waypoints",frames=(0.2, 0.2, 0.6),ground=-0.05)

In [ ]:
green_paths = []
for sysid,path in green_gcs.paths.items():
    green_paths.append(np.array([x for x in path if x is not None]))

markers = [ConfigGazebo.create_trajectory_from_array(array=path,color=Color.GREEN) for path in green_paths]
Gazebo.plot_3d_interactive(markers=markers,title="Simulation Waypoints",frames=(0.2, 0.2, 0.6),ground=-0.05)